# data analysis for paid child leave, gdp growth, Data cost
- # women empowerment index data (df)
- for 190 countries over 50 years, 1971-2020. 
- There are 35 yes or no questions asked each year in 8 categories:
[Assets
Entrepreneurship
Marriage
Mobility
Parenthood
Pay
Pension
Workplace]

- (yes) answers are worth either 20 or 25 points
- no country experiences negative growth in the index over 50 years
- data can be found here:
https://data.world/makeovermonday/2020w26


- # gdp growth data (gdp)
- for 266 countries over ~60 years
- The YoY growth is on a relative basis
- Data should be reduced to 190 Countries for ease of comparison
- data can be found here:
https://www.kaggle.com/datasets/zackerym/gdp-annual-growth-for-each-country-1960-2020?select=GDP_annual_growth.csv

- # data cost (gig) 
- Data for average cost per gigabyte in 2020
- in USD
- dataset can be found here:
https://data.world/makeovermonday/2021w19

- # gapminder data (gapminder)
- population, life expectancy data, gdp per cap
- data available through plotly:
-
(import plotly.express as px

#get country health, population, gdpPercap data from gapminder through plotly
df2 = px.data.gapminder().query('year > 1971')
gap = df2[['country', 'year', 'lifeExp', 'pop', 'gdpPercap']].copy().reset_index(drop=True)
gap['country'].unique().tolist()
#gap.to_csv('/Users/amritambe/Desktop/Analysis_Project/Women_Empowerment/2 Data/Gapminder/gapminder_pop_data.csv', index=False)
)


In [2]:
import pandas as pd
import numpy as np
import os

path = r'/Users/amritambe/Desktop/Analysis_Project/Women_Empowerment/2 Data'

#read in paid child leave data
df = pd.read_csv(os.path.join(path, 'Empowerment_Index', 'paid_leave_qs.csv'))
df1 = pd.read_csv(os.path.join(path, 'Empowerment_Index', 'Country_50y_change.csv'))
df2 = pd.read_csv(os.path.join(path, 'Empowerment_Index', 'All_Qs.csv'))

#read in gdp data
gdp = pd.read_csv(os.path.join(path, 'GDP_Growth', 'GDP_annual_growth_NEW.csv'))

#read in data cost df
gig = pd.read_csv(os.path.join(path, 'Technology', 'Gb_Cost_World.csv'))

#read in gapminder data
gapminder = pd.read_csv(os.path.join(path, 'Gapminder', 'gapminder_pop_data.csv'))


#print checks
print(f'The dataframes are of shapes: \n\
df (paid leave Qs):{df.shape}, \n\
df1 (50 years change):{df1.shape}, \n\
df2 (All Qs): {df2.shape}, \n\
gig (cost of data): {gig.shape}, \n\
gdp (gdp data): {gdp.shape}, \n\
gapminder (pop data): {gapminder.shape}')

The dataframes are of shapes: 
df (paid leave Qs):(28500, 4), 
df1 (50 years change):(190, 4), 
df2 (All Qs): (332500, 7), 
gig (cost of data): (155, 3), 
gdp (gdp data): (266, 66), 
gapminder (pop data): (1136, 5)






# transforming gdp
firstly, gdp needs transformed because each year has its own column. 
we need each country to have muliple years, and have one column for all years.

In [3]:
#the gdp dataset needs to be transformed in order to be similar to the parental leave data
#pd.set_option('display.max_rows', None)

#gdp.isnull().sum()

#gdp.columns.tolist()

''' pd.melt()
use pd.melt(df, id_vars=variables to keep intact, 
                var_name=variables to melt, 
                value_name=what is melted value to be named)
'''

melted_gdp = pd.melt(gdp, 
                    id_vars=['Country Name', 'Country Code',  'Indicator Name', 'Indicator Code'],
                    var_name='Year', 
                    value_name='GDP Growth')

#sort
sorted_gdp = melted_gdp.sort_values(by=['Country Name', 'Year'], ascending=[True, True])

#df and df1 start at 1971, so gdp should start at 71 as well
gdp_clean = sorted_gdp.query('Year >= "1971"').copy()

#check
print(f'Eliminated {sorted_gdp.shape[0]-gdp_clean.shape[0]} rows of data before 1971, GDP data has shape: {gdp_clean.shape}')


#change columns to replace spaces with underscores
gdp_clean.columns = ['Country_Name', 'Country_Code', 'Indicator_Name', 'Indicator_Code',
       'Year', 'GDP_Growth']

gdp_clean.columns

#check gdp_clean for null values
null_gdp = gdp_clean.groupby('Country_Name')['GDP_Growth'] \
    .apply(lambda x: x.isnull() \
        .sum()) \
            .reset_index() \
                .sort_values(by='GDP_Growth', ascending=False)

#rename columns
null_gdp.columns = ['Country_Name', 'GDP_Nulls']

#get list of all Countries with n+ NULLs for GDP growth.
#there are 266 countries, 50 years of data. 

n = 100

null_countries = null_gdp.query('GDP_Nulls >= @n')['Country_Name'].copy().tolist()

#delete high null countries
gdp_cleaned = gdp_clean.query('Country_Name not in @null_countries')


# PRINT CHECKS
print(f'{len(null_countries)} countries were deleted, resulting in {gdp_clean.shape[0]-gdp_cleaned.shape[0]} fewer rows. GDP data has shape: {gdp_cleaned.shape}')
#check percentage of years > 1% gdp growth
print(f"{(gdp_cleaned.query('GDP_Growth > 1').sort_values(by=['GDP_Growth'], ascending=[False]).shape[0] / gdp_cleaned.shape[0] * 100) :0.2f}% of {gdp_cleaned['Year'].nunique()} total years saw growth over 1%")
#check number of years with negative gdp growth
print(f"{gdp_cleaned.query('GDP_Growth < 0').shape[0] / gdp_cleaned.shape[0] * 100:0.2f}% of years saw negative gdp growth")

Eliminated 2926 rows of data before 1971, GDP data has shape: (13566, 6)
0 countries were deleted, resulting in 0 fewer rows. GDP data has shape: (13566, 6)
61.93% of 51 total years saw growth over 1%
12.91% of years saw negative gdp growth



# explore datasets and change column names

In [5]:
gapminder.columns = ['Country_Name', 'Year', 'LifeExp', 'Population', 'GDP_Per_Cap']
gapminder.columns.tolist()

['Country_Name', 'Year', 'LifeExp', 'Population', 'GDP_Per_Cap']

In [6]:
#check df, change column names for paid leave dataframes

df.columns = ['Country_Name', 'Year', 'Questions', 'Score']

df1.columns = ['Country_Name', 'Index_1971', 'Index_2020', 'Fifty_Year_Change']

df2.columns = ['Country_Name',
  'Region',
  'Income_group',
  'Year',
  'Question_Category',
  'Question',
  'Score']

gapminder.columns = ['Country_Name', 'Year', 'LifeExp', 'Population', 'GDP_Per_Cap']

# Mobile data cost
#check gig.columns
gig.columns = ['Rank', 'Country_Name', '1GB_Price_Usd']
gig.columns

df.columns, df1.columns, df2.columns

#df.isnull().sum(), df1.isnull().sum()


#check number of countries in all 3 df
print(f"df # of countries: {df['Country_Name'].nunique()}\n\
df1 # of countries: {df1['Country_Name'].nunique()}\n\
df2 # of countries: {df2['Country_Name'].nunique()}\n\
gig # of countries: {gig['Country_Name'].nunique()} \n\
gdp_cleaned # of countries: {gdp_cleaned['Country_Name'].nunique()}\n\
gapminder # of countries: {gapminder['Country_Name'].nunique()}")

# #check df and df1 are same
print(f"df and df1 Countries are exactly the same: {df['Country_Name'].unique().tolist() == df1['Country_Name'].unique().tolist()}")



df # of countries: 190
df1 # of countries: 190
df2 # of countries: 190
gig # of countries: 155 
gdp_cleaned # of countries: 266
gapminder # of countries: 142
df and df1 Countries are exactly the same: False


# explore difference between Country Name data

In [22]:
#make lists to analyze country names 
df_c = df['Country_Name'].unique().tolist()
df1_c = df1['Country_Name'].unique().tolist()
df2_c = df2['Country_Name'].unique().tolist()
gig_c = gig['Country_Name'].unique().tolist()
gdp_c = gdp_cleaned['Country_Name'].unique().tolist()
gap_c = gapminder['Country_Name'].unique().tolist()

In [69]:
def find_mismatched_countries(list1, list2):
    common_countries = [country for country in list1 if country in list2]
    print(f'There are {len(common_countries)} common countries between list1({len(list1)}) and list2({len(list2)}).')

    set1 = set(list1)
    set2 = set(list2)

    different_countries = set1 - set2
    list_different = list(different_countries)

    print(f'There are {len(different_countries)} mismatched countries:')
    return list_different


# WE Index data

In [70]:
#use a list comprehension to find common countries
find_mismatched_countries(df_c, df1_c)
print('\nThe first false must be because of whitespace or capitalization. these should work as-is in tableau for visualizations.')

There are 190 common countries between list1(190) and list2(190).
There are 0 mismatched countries:

The first false must be because of whitespace or capitalization. these should work as-is in tableau for visualizations.


# data cost 

In [71]:
# identify which countries dont match from the gig dataframe and the rest
find_mismatched_countries(gig_c, df1_c)

There are 136 common countries between list1(155) and list2(190).
There are 19 mismatched countries:


['Taiwan',
 'Slovakia',
 'Puerto Rico',
 'Yemen',
 'Syria',
 'Western Sahara',
 'Turkmenistan',
 'Congo',
 'Cuba',
 "Côte d'Ivoire",
 'Macedonia',
 'Palestine',
 'Hong Kong',
 'Iran',
 'Monaco',
 'Egypt',
 'Gambia',
 'South Korea',
 'Kyrgyzstan']

In [73]:
df1.query('Country_Name.str.contains("Egy")')

,Country_Name,Index_1971,Index_2020,Fifty_Year_Change
152,"Egypt, Arab Rep.",235,360,125


# gdp data

In [41]:
#use a list comprehension to find common countries for gdp_cleaned
find_mismatched_countries(gdp_c, df1_c)

There are 187 common countries between list1(266) and list2(190).
There are 79 mismatched countries:


['IDA total',
 'Macao SAR, China',
 'Turks and Caicos Islands',
 'Latin America & the Caribbean (IDA & IBRD countries)',
 'Puerto Rico',
 'New Caledonia',
 'Sub-Saharan Africa (IDA & IBRD countries)',
 'Small states',
 'Gibraltar',
 'Early-demographic dividend',
 'Greenland',
 'Nauru',
 'North America',
 'Middle East & North Africa (excluding high income)',
 'Aruba',
 'Central Europe and the Baltics',
 'Andorra',
 'Post-demographic dividend',
 'High income',
 'Lower middle income',
 'Pacific island small states',
 'Bermuda',
 'Fragile and conflict affected situations',
 'French Polynesia',
 'Upper middle income',
 'European Union',
 'Cayman Islands',
 'OECD members',
 'Guam',
 'Latin America & Caribbean',
 'Isle of Man',
 "Cote d'Ivoire",
 'World',
 'Turkmenistan',
 'Cuba',
 'Caribbean small states',
 'South Asia (IDA & IBRD)',
 'Faroe Islands',
 'Sub-Saharan Africa (excluding high income)',
 'Middle East & North Africa (IDA & IBRD countries)',
 'Northern Mariana Islands',
 'Other smal

# gapminder data

In [42]:
#check countries between gap and df2
find_mismatched_countries(gap_c, df2_c)

There are 129 common countries between list1(142) and list2(190).
There are 13 mismatched countries:


['Korea, Dem. Rep.',
 "Cote d'Ivoire",
 'Iran',
 'Swaziland',
 'Taiwan',
 'Venezuela',
 'Egypt',
 'Reunion',
 'Puerto Rico',
 'Cuba',
 'Gambia',
 'Hong Kong, China',
 'Syria']

In [43]:
#PUERTO RICO (U.S)
#change df, df1, df2 to 'Puerto Rico'
# df['Country_Name'] = df['Country_Name'].replace({'Puerto Rico (U.S.)':'Puerto Rico'})
# df1['Country_Name'] = df1['Country_Name'].replace({'Puerto Rico (U.S.)':'Puerto Rico'})
# df2['Country_Name'] = df2['Country_Name'].replace({'Puerto Rico (U.S.)':'Puerto Rico'})

#df2.query('Country_Name.str.contains("Puerto Rico")').head()


#COTE D'IVOIRE
#change Cote d'Ivoire	to ivory coast
#gdp_cleaned['Country_Name'] = gdp_cleaned['Country_Name'].replace({"Cote d'Ivoire":"Ivory Coast"})

# export cleaned data


In [45]:
# df2 = df2.drop(columns='index', axis=1)
gdp_cleaned = gdp_cleaned.drop(columns=['Country_Code', 'Indicator_Name', 'Indicator_Code'], axis=1)
gdp_cleaned.columns

Index(['Country_Name', 'Year', 'GDP_Growth'], dtype='object')

In [47]:
gdp_cleaned.shape, df.shape, df1.shape, df2.shape, gig.shape, gapminder.shape

((13566, 3), (28500, 4), (190, 4), (332500, 7), (155, 3), (1136, 5))

# Merging 

In [53]:
gdp_cleaned = gdp_cleaned[gdp_cleaned['Year'] != "Unnamed: 65"].copy()
gdp_cleaned['Year'] = gdp_cleaned['Year'].astype('int')

In [63]:
merged_df = pd.merge(gdp_cleaned, df, on=['Country_Name', 'Year'], how='outer')
merged_df = pd.merge(merged_df, df1, on=['Country_Name'], how='outer')
merged_df = pd.merge(merged_df, df2, on=['Country_Name', 'Year'], how='outer')
merged_df = pd.merge(merged_df, gig, on=['Country_Name'], how='outer')
merged_df = pd.merge(merged_df, gapminder, on=['Country_Name', 'Year'], how='outer')
merged_df.reset_index(drop=True)
merged_df.shape

(1001545, 18)

In [74]:
# merged_df = merged_df.drop(columns=['Questions', 'Score_x'], axis=1).copy()
# merged_df = merged_df.drop_duplicates().copy()
# merged_df.query('Country_Name == "Afghanistan" and Year == 1971 and Question.str.contains("Do female and male surviving spouses")', engine='python')
merged_df.head()

,Country_Name,Year,GDP_Growth,Index_1971,Index_2020,Fifty_Year_Change,Region,Income_group,Question_Category,Question,Score_y,Rank,1GB_Price_Usd,LifeExp,Population,GDP_Per_Cap
0,Afghanistan,1971.0,NaN,210.0,305.0,95.0,South Asia,Low income,Assets,Do female and male surviving spouses have equa...,0.0,59.0,1.55,NaN,NaN,NaN
1,Afghanistan,1971.0,NaN,210.0,305.0,95.0,South Asia,Low income,Assets,Do men and women have equal ownership rights t...,20.0,59.0,1.55,NaN,NaN,NaN
2,Afghanistan,1971.0,NaN,210.0,305.0,95.0,South Asia,Low income,Assets,Do sons and daughters have equal rights to inh...,0.0,59.0,1.55,NaN,NaN,NaN
3,Afghanistan,1971.0,NaN,210.0,305.0,95.0,South Asia,Low income,Assets,Does the law grant spouses equal administrativ...,20.0,59.0,1.55,NaN,NaN,NaN
4,Afghanistan,1971.0,NaN,210.0,305.0,95.0,South Asia,Low income,Assets,Does the law provide for the valuation of nonm...,0.0,59.0,1.55,NaN,NaN,NaN


In [68]:
#export
# df.to_csv(os.path.join(path, 'Clean_Data', 'Child_Leave_Qs_Clean.csv'))
# df1.to_csv(os.path.join(path, 'Clean_Data', 'Fifty_Year_Change_Clean.csv'))
# df2.to_csv(os.path.join(path, 'Clean_Data', 'All_Questions_Clean.csv'))
# gdp_cleaned.to_csv(os.path.join(path, 'Clean_Data', 'GDP_Clean.csv'))
# merged_df.to_csv(os.path.join(path, 'Clean_Data', '11_10_2023_WEI.csv'))

In [31]:
print(f'Women Empowerment Descriptive Statistics.\n\
Child_Leave_Qs_Clean.csv: \n\
{df.describe()}')

Women Empowerment Descriptive Statistics.
Child_Leave_Qs_Clean.csv: 
               Year         Score
count  28500.000000  28500.000000
mean    1995.500000      4.702456
std       14.431123      8.481660
min     1971.000000      0.000000
25%     1983.000000      0.000000
50%     1995.500000      0.000000
75%     2008.000000      0.000000
max     2020.000000     20.000000


In [37]:
print(f'Women Empowerment Descriptive Statistics.\n\
Fifty_Year_Change_Clean.csv: \n\
{df1.describe()}')

Women Empowerment Descriptive Statistics.
Fifty_Year_Change_Clean.csv: 
       Index_1971  Index_2020  Fifty_Year_Change
count  190.000000  190.000000         190.000000
mean   371.105263  601.842105         230.736842
std    101.296304  138.505921         112.987260
min    140.000000  210.000000           0.000000
25%    296.250000  522.500000         155.000000
50%    385.000000  630.000000         225.000000
75%    443.750000  695.000000         298.750000
max    570.000000  800.000000         540.000000


In [35]:
print(f'Women Empowerment Descriptive Statistics.\n\
All_Questions_Clean.csv: \n\
{df2.describe()}')

Women Empowerment Descriptive Statistics.
All_Questions_Clean.csv: 
                Year          Score
count  332500.000000  332500.000000
mean     1995.500000      13.511053
std        14.430891      11.437948
min      1971.000000       0.000000
25%      1983.000000       0.000000
50%      1995.500000      20.000000
75%      2008.000000      25.000000
max      2020.000000      25.000000


In [36]:
print(f'GDP Growth Descriptive Statistics.\n\
GDP_Clean.csv: \n\
{df.describe()}')

GDP Growth Descriptive Statistics.
GDP_Clean.csv: 
               Year         Score
count  28500.000000  28500.000000
mean    1995.500000      4.702456
std       14.431123      8.481660
min     1971.000000      0.000000
25%     1983.000000      0.000000
50%     1995.500000      0.000000
75%     2008.000000      0.000000
max     2020.000000     20.000000
